<a href="https://colab.research.google.com/github/royd2023/Linear-Regression-Health-Costs/blob/main/fcc_predict_health_costs_with_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# One-hot encode categorical features
dataset = pd.get_dummies(dataset, columns=['sex', 'smoker', 'region'])

In [ ]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)

train_labels = train_dataset.pop("expenses")
test_labels = test_dataset.pop("expenses")

In [ ]:
# Normalize numeric features
numeric_features = ['age', 'bmi', 'children']
train_dataset[numeric_features] = (train_dataset[numeric_features] - train_dataset[numeric_features].mean()) / train_dataset[numeric_features].std()
test_dataset[numeric_features] = (test_dataset[numeric_features] - test_dataset[numeric_features].mean()) / test_dataset[numeric_features].std()


In [ ]:
# input function

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  def input_function():
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))
    if shuffle:
      ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(num_epochs)
    return ds
  return input_function

train_input_fn = make_input_fn(train_dataset, train_labels)
test_input_fn = make_input_fn(test_dataset, test_labels, num_epochs=1, shuffle=False)

In [ ]:
# Define the model
model = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=[len(train_dataset.columns)]),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1)  # output layer for regression
])

# Compile the model
model.compile(
    optimizer='adam',
    loss='mse',           # Mean Squared Error for regression
    metrics=['mae', 'mse']       # Mean Absolute Error as an additional metric
)

In [ ]:
# train the model

model.fit(train_dataset, train_labels, epochs=100, validation_split=0.2)
result = model.evaluate(test_dataset, test_labels)
print("Test Loss:", result[0])
print("Test MAE:", result[1])  # or accuracy if you're doing classification

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
